<a href="https://colab.research.google.com/github/giordamaug/LION15_Experiments/blob/main/notebook/PSCN_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PSCN experiments as embedding method (LION15 SI)
In this notebook we implement a Patchy-San Convolutional Network (PSCN) [1] algorithm.

The Patchy-San implememntation uses the [Networkx](https://networkx.org/) Library for graph management, and [Keras](https://keras.io/) (Tensorflow) for deep learning library.

A new data loader for PSCN has been implemented in order to input graph files from graphml format to Networkx library.

**References**

[1] Learning Convolutional Neural Networks for Graphs,
Mathias Niepert, Mohamed Ahmed, Konstantin Kutzkov, Proceedings of The 33rd International Conference on Machine Learning, PMLR 48:2014-2023, 2016. [link](http://proceedings.mlr.press/v48/niepert16)

## Download the PSCN (github) software 
PSCN software is downloaded and installed in the colab by means of `git` utility. The software requires that an additional python package (`pynauty`) to be compiled and installed. 

In [ ]:
!git clone https://github.com/tvayer/PSCN.git
%cd '/content/PSCN/pynauty-0.6.0'
!make pynauty
!make user-ins pynauty
!pip install .

     |████████████████████████████████| 3.2 MB 5.3 MB/s 


## Cloning Github repository 
In this code snipped we make oa copy of LION15_Experiments GitHu b repository. Required for downloading graph datasets on google colab notebook.

In [ ]:
!git clone https://github.com/giordamaug/LION15_Experiments.git
%cd /content/LION15_Experiments/notebook

## Import all required packages

In [29]:
import warnings
warnings.filterwarnings("ignore")
import networkx as nx
import operator
import random
import pandas as pd
import numpy as np
import random
import os
import tqdm as tq
import warnings
import sys
if 'pscn' in sys.modules.keys(): 
  del sys.modules["pscn"]
from pscn import PSCN
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,matthews_corrcoef,accuracy_score,precision_score,f1_score, recall_score
import tqdm.notebook as tq
import os

## The edge attack routine
Currently two strategies have been implemneted for edge removal attacks:
1. random selection
2. based on betweness centrality of edges

The routine accepts as input the removal criteria (`random`, `betweeness`), the amount (in `percentage`) of edges to remove, and a random seed (for reprucibility of experiments)

In [19]:
def nx_edgeattack(G: nx.Graph, criteria = "random", percentage=30, verbose=False, random_state=42):
  at = percentage/100.0
  #remove_zero_weights(G)
  if criteria == "betweeness":
    score = nx.edge_betweenness(G).items()
  elif criteria == "degree":
    raise Exception("Wrong criteria")
  elif criteria == "random":
    score = list(G.edges())
    random.Random(random_state).shuffle(score)
    score = list(dict(zip(score,range(len(score)))).items())
  else:
    raise Exception("Wrong criteria")
  edges_to_remove = sorted(score, key=operator.itemgetter(1, 0), reverse=True)[0:int(len(score)*at)]
  #assert len(edges_to_remove) > 0, "Nothing to remove!"
  for e,w in edges_to_remove:
    G.remove_edge(e[0], e[1])
  if verbose:
    print("removed", edges_to_remove)
  return 0,len(edges_to_remove)

## The dataset loading function
The dataset of graphs in (`graphml` formats) is converted into the [igraph](https://igraph.org/python/doc/tutorial/tutorial.html) library data structures. The loading procedure (`load_graphs`) generates two copies of the dataset: the first is the original graph set, while the second is the dataset modified by the attacking routine (edge removal). The third argument is a pandas dataframe containing for each graph name the associated label (used for classification/training).

In [22]:
def load_dataset(dataset, root_dir='.', verbose=False, use_deg_label=False):
    with open(f'{root_dir}/{dataset}/{dataset}_graph_indicator.txt', "r") as f:
        graph_indicator = [int(i) - 1 for i in list(f)]
    f.closed

    # Nodes.
    self.num_graphs = max(graph_indicator)
    node_indices = []
    offset = []
    c = 0

    for i in (tqdm(range(num_graphs + 1), desc="Getting node indices:") if verbose else range(self.num_graphs + 1)):
        offset.append(c)
        c_i = graph_indicator.count(i)
        node_indices.append((c, c + c_i - 1))
        c += c_i

    graph_db = []
    gc = 1
    for i in (tqdm(node_indices, desc="Loading nodes:") if verbose else node_indices):
        g = nx.Graph()
        g.graph['name'] = f'{dataset}_{gc}'
        gc += 1
        for j in range(i[1] - i[0]+1):
            g.add_node(j)

        graph_db.append(g)

    # Edges.
    with open(f'{root_dir}/{dataset}/{dataset}_A.txt', "r") as f:
        edges = [i.split(',') for i in list(f)]
    f.closed
    edges = [(int(e[0].strip()) - 1, int(e[1].strip()) - 1) for e in edges]
    edge_list = []
    edgeb_list = []
    for e in (tqdm(edges, desc="Loading edges:")if verbose else edges):
        g_id = graph_indicator[e[0]]
        g = graph_db[g_id]
        off = offset[g_id]

        # Avoid multigraph (for edge_list)
        if ((e[0] - off, e[1] - off) not in list(g.edges())) and ((e[1] - off, e[0] - off) not in list(g.edges())):
            g.add_edge(e[0] - off, e[1] - off)
            edge_list.append((e[0] - off, e[1] - off))
            edgeb_list.append(True)
        else:
            edgeb_list.append(False)

    # Node labels.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_node_labels.txt'):
        print("Loading node labels...") if self.verbose else None
        with open(f'{root_dir}/{dataset}/{dataset}_node_labels.txt', "r") as f:
            node_labels = [str.strip(i) for i in list(f)]
        f.closed
        
        # multiple node labels (not supported!)
        #node_labels = [i.split(',') for i in node_labels]
        #int_labels = [];
        #for i in range(len(node_labels)):
        #    int_labels.append([int(j) for j in node_labels[i]])
        
        i = 0
        for g in graph_db:
            for v in range(g.number_of_nodes()):
                g.nodes[v]['label'] = int(node_labels[i])
                i += 1
    else:
        if self.use_deg_label:
            print("Calculating node labels as degree...") if self.verbose else None
            i = 0
            for g in graph_db:
                for v in range(g.number_of_nodes()):
                    g.nodes[v]['label'] = g.degree(v)
                    i += 1

    # Node Attributes.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_node_attributes.txt'):
        print("Loading node attributes...") if self.verbose else None
        with open(os.path.join(f'{root_dir}/{dataset}/{dataset}_node_attributes.txt', "r") as f:
            node_attributes = [str.strip(i) for i in list(f)]
        f.closed
        
        node_attributes = [i.split(',') for i in node_attributes]
        float_attributes = [];
        for i in range(len(node_attributes)):
            float_attributes.append([float(j) for j in node_attributes[i]])
        #maxLength = max(len(x) for x in float_attributes )
        #if len(maxLength) > 0:
        i = 0
        for g in graph_db:
            for v in range(g.number_of_nodes()):
                g.nodes[v]['attributes'] = float_attributes[i]
                i += 1
    # Edge Labels.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_edge_labels.txt'):
        print("Loading edge labels...") if self.verbose else None
        with open(f'{root_dir}/{dataset}/{dataset}_edge_labels.txt', "r") as f:
            edge_labels = [str.strip(i) for i in list(f)]
        f.closed

        #edge_labels = [i.split(',') for i in edge_labels]
        e_labels = []
        for i in range(len(edge_labels)):
            if(edgeb_list[i]):
                e_labels.append([int(j) for j in edge_labels[i]])
        
        i = 0
        for g in graph_db:
            for e in range(g.number_of_edges()):
                g.edges[edge_list[i]]['label'] = int(edge_labels[i])
                i += 1

    # Edge Attributes.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_edge_attributes.txt'):
        print("Loading edge attributes...") if self.verbose else None
        with open('{root_dir}/{dataset}/{dataset}_edge_attributes.txt', "r") as f:
            edge_attributes = [str.strip(i) for i in list(f)]
        f.closed

        edge_attributes = [i.split(',') for i in edge_attributes]
        e_attributes = []
        for i in range(len(edge_attributes)):
            if(edgeb_list[i]):
                e_attributes.append([float(j) for j in edge_attributes[i]])
        
        i = 0
        for g in graph_db:
            for e in range(g.number_of_edges()):
                g.edges[edge_list[i]]['attributes'] = e_attributes[i]
                i += 1

    # Classes.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_graph_labels.txt'):
        print("Loading graph labels...") if self.verbose else None
        with open(f'{root_dir}/{dataset}/{dataset}_graph_labels.txt', "r") as f:
            classes = [str.strip(i) for i in list(f)]
        f.closed
        #classes = [i.split(',') for i in classes]
        #cs = [];
        #for i in range(len(classes)):
        #    cs.append([int(j) for j in classes[i]])        
        i = 0
        for g in graph_db:
            g.graph['class'] = int(classes[i])
            i += 1

    # Targets.
    if os.path.exists(f'{root_dir}/{dataset}/{dataset}_graph_attributes.txt'):
        print("Loading graph attributes...") if self.verbose else None
        with open(f'{root_dir}/{dataset}/{dataset}_graph_attributes.txt', 'r') as f:
            targets = [str.strip(i) for i in list(f)]
        f.closed
        
        targets= [i.split(',') for i in targets]
        ts = [];
        for i in range(len(targets)):
            ts.append([float(j) for j in targets[i]])
        
        i = 0
        for g in graph_db:
            g.graph['attributes'] = ts[i]
            i += 1
    # Generate label file
    labels = []
    for g in graph_db:
        labels.append(g.graph['class'])
    return graph_db, labels

## Load the dataset from TUD format
This code snippet loads the dataset of graphs (in `graphml` format) as well as it performs graph attacks (see the above defined `load_graph` procedure. At the end a summary of graph modifications is printed out.



In [41]:
dataname = 'PROTEINS' #@param ['MUTAG', 'KIDNEY', 'PROTEINS']
criteria = 'random' #@param ['random', 'betweeness']
percentage = 10 #@param [0, 5, 10, 20, 30, 40, 50] {type:"raw"}
ontest = False #@param {type:"boolean"}
shutil.unpack_archive(f'../datasets/{dataname}.zip', '../datasets/')
graphs, y = load_dataset(dataname, root_dir='../datasets')
graphs_adv = []  # no attack if dataset is loaded from TU format
for G in tq.tqdm(graphs, desc="Copy/Attack"):
  Gadv = G.copy()
  ec = Gadv.ecount()
  if percentage > 0: 
    n,e = ig_edgeattack(Gadv, criteria=criteria, percentage=percentage, random_state=42)
  graphs_adv += [Gadv]

nclasses = len(set(y))
print("No Classes: %d\n"%nclasses)
summary = pd.DataFrame(
    [(g.vcount(), g.ecount(),ga.vcount(), ga.ecount()) for g,ga in zip(graphs,graphs_adv)],
    columns=["(Graph) nodes", "(Graph) edges", "(Attacked Graph) nodes", "(Attacked Graph) edges"],
)
summary.describe().round(2)

Copy/Attack: 100%|██████████| 1113/1113 [00:00<00:00, 10654.48it/s]

No Classes: 2



,(Graph) nodes,(Graph) edges,(Attacked Graph) nodes,(Attacked Graph) edges
count,1113.00,1113.00,1113.00,1113.00
mean,39.06,72.82,39.06,65.99
std,45.78,84.64,45.78,76.15
min,4.00,5.00,4.00,5.00
25%,15.00,28.00,15.00,26.00
50%,26.00,49.00,26.00,45.00
75%,45.00,87.00,45.00,79.00
max,620.00,1049.00,620.00,945.00


## The Evaluation Pipeline
Thsi is the main part of the experiment. Once data have been loaded and modified (by the attacking routine), we carry on a stratified 10-fold cross-validation of the pipeline consisting in the iNP2V embedding model plus a SVM linear kernel classifier.

In each fold a iNP2V model is trained on 90% of the dataset; the so-trained model is used to embed both the training and the testing samples (inductively). Then the embedding arrays are used as training/testing inputs of the SVM classifier. We collect predictions results for all folds, and the mean and standard deviation of several metric are printed out (Accuracy, Precision, F-measure, Recall and Matthews Correlation Coefficients). 

In [42]:

# Load model parameters
params = {"agg_by": [1], "cut_off": [0.1], "dimensions": 512, "encodew": False, "epochs": 100, 
    "extractor": [1], "min_count": 3, "prob_type": ["ndd"], "save_vocab": False, "seed": 1, "verbose": False, 
    "vertex_attribute": "label", "workers": 4}
start = time()
G = np.array(graphs)
Gadv = np.array(graphs_adv)
y = np.array(y)
cv_folds = 10 #@param {type:"slider", min:2, max:10, step:1}
tot_preds = np.array([])
tot_targets = np.array([])
tot_acc = np.array([])
tot_prec = np.array([])
tot_F1 = np.array([])
tot_recall = np.array([])
tot_MCC = np.array([])
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
for train_index, test_index in tq.tqdm(list(skf.split(G,y)), desc="fold: "):
    G_train, G_test = G[train_index], Gadv[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = Netpro2vec(**params)
    X_train = model.fit(G_train).get_embedding()
    X_test = np.array(model.infer_vector(G_test))
    y_pred = SVC(kernel='linear').fit(X_train,y_train).predict(X_test)
    tot_preds = np.append(tot_preds,y_pred)
    tot_targets = np.append(tot_targets,y_test)
    tot_acc = np.append(tot_acc, accuracy_score(y_test, y_pred))
    tot_prec = np.append(tot_prec, precision_score(y_test, y_pred, average='macro'))
    tot_F1 = np.append(tot_F1, f1_score(y_test, y_pred, average='macro'))
    tot_recall = np.append(tot_recall, recall_score(y_test, y_pred, average='macro'))
    tot_MCC = np.append(tot_MCC, matthews_corrcoef(y_test, y_pred))
temp = time() - start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
expired = '%d:%d:%d' %(hours,minutes,seconds)
print()
print(confusion_matrix(tot_targets, tot_preds))
print("Acc\t%.2f\u00B1%.2f"%((tot_acc * 100).mean(), (tot_acc * 100).std()))
print("Prec\t%.2f\u00B1%.2f"%(tot_prec.mean(), tot_prec.std()))
print("F1\t%.2f\u00B1%.2f"%(tot_F1.mean(), tot_F1.std()))
print("Recall\t%.2f\u00B1%.2f"%(tot_recall.mean(), tot_recall.std()))
print('MCC\t%.2f\u00B1%.2f'%(tot_MCC.mean(), tot_MCC.std()))
print('Elapsed time', expired)

fold: 100%|██████████| 10/10 [01:01<00:00,  6.13s/it]


[[439 224]
 [166 284]]
Acc	64.95±4.91
Prec	0.64±0.05
F1	0.64±0.05
Recall	0.65±0.05
MCC	0.29±0.09
Elapsed time 0:1:1
